# 改善版：競馬予測モデルと高度な賭け方戦略

1. モデルの改善（閾値調整、特徴量追加）
2. 期待値ベースの賭け方戦略
3. 3連単・3連複を含む複合的な賭け方

In [ ]:
import sys
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve
import ast
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

# Use default font (no Japanese font needed)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

## 1. 改善されたモデルクラス

In [28]:
class ImprovedKeibaPredictor:
    """改善版競馬予測モデル"""
    
    def __init__(self):
        self.models = []  # アンサンブル用の複数モデル
        self.feature_importance_df = None
        self.label_encoders = {}  # カテゴリカル特徴量のエンコーダー
        
    def create_additional_features(self, df):
        """追加の特徴量を作成"""
        df = df.copy()
        
        # 1. 馬の調子指標（過去3走の着順変化）
        if all(f'着順{i}' in df.columns for i in range(1, 4)):
            df['着順トレンド'] = (df['着順1'] - df['着順3']) / 3
            df['直近好調度'] = (df['着順1'] <= 3).astype(int) + (df['着順2'] <= 3).astype(int)
        
        # 2. 騎手×馬場の相性
        if '騎手' in df.columns and '芝・ダート' in df.columns:
            df['騎手_馬場'] = df['騎手'].astype(str) + '_' + df['芝・ダート'].astype(str)
            
        # 3. 距離適性指標
        if all(f'距離{i}' in df.columns for i in range(1, 4)) and '距離' in df.columns:
            past_distances = df[['距離1', '距離2', '距離3']]
            df['距離適性'] = 1 / (1 + np.abs(df['距離'] - past_distances.mean(axis=1)))
            
        # 4. 休養明け指標
        if '日付差' in df.columns:
            df['休養明け'] = (df['日付差'] > 60).astype(int)
            df['適正休養'] = ((df['日付差'] >= 14) & (df['日付差'] <= 35)).astype(int)
            
        # 5. 重賞実績
        if all(f'クラス{i}' in df.columns for i in range(1, 4)):
            df['重賞実績'] = ((df['クラス1'] >= 8) | (df['クラス2'] >= 8) | (df['クラス3'] >= 8)).astype(int)
            
        # 6. 斤量変化の影響
        if '斤量' in df.columns and '斤量1' in df.columns:
            df['斤量増'] = df['斤量'] - df['斤量1']
            
        return df
    
    def encode_categorical_features(self, X_train, X_valid=None):
        """カテゴリカル特徴量をエンコード"""
        from sklearn.preprocessing import LabelEncoder
        
        X_train_encoded = X_train.copy()
        X_valid_encoded = X_valid.copy() if X_valid is not None else None
        
        # カテゴリカル特徴量を検出（object型の列）
        categorical_cols = X_train_encoded.select_dtypes(include=['object']).columns
        
        for col in categorical_cols:
            if col not in self.label_encoders:
                self.label_encoders[col] = LabelEncoder()
                # 訓練データでフィット
                self.label_encoders[col].fit(X_train_encoded[col].fillna('missing'))
            
            # 訓練データをエンコード
            X_train_encoded[col] = self.label_encoders[col].transform(X_train_encoded[col].fillna('missing'))
            
            # 検証データをエンコード
            if X_valid_encoded is not None:
                # 未知のカテゴリは-1に置換
                valid_values = X_valid_encoded[col].fillna('missing')
                valid_encoded = []
                for val in valid_values:
                    if val in self.label_encoders[col].classes_:
                        valid_encoded.append(self.label_encoders[col].transform([val])[0])
                    else:
                        valid_encoded.append(-1)  # 未知のカテゴリ
                X_valid_encoded[col] = valid_encoded
        
        return X_train_encoded, X_valid_encoded
    
    def train_ensemble(self, X_train, y_train, X_valid, y_valid, n_models=3):
        """アンサンブルモデルの学習"""
        self.models = []
        
        # カテゴリカル特徴量をエンコード
        print("カテゴリカル特徴量をエンコード中...")
        X_train_encoded, X_valid_encoded = self.encode_categorical_features(X_train, X_valid)
        
        # 異なるパラメータで複数モデルを学習
        base_params = {
            'objective': 'binary',
            'metric': 'binary_logloss',
            'verbosity': -1,
            'random_state': 42
        }
        
        param_variations = [
            {'num_leaves': 31, 'learning_rate': 0.05, 'feature_fraction': 0.9},
            {'num_leaves': 50, 'learning_rate': 0.03, 'feature_fraction': 0.8},
            {'num_leaves': 20, 'learning_rate': 0.1, 'feature_fraction': 0.7}
        ]
        
        valid_predictions = []
        
        for i, params in enumerate(param_variations[:n_models]):
            print(f"\nモデル{i+1}を学習中...")
            full_params = {**base_params, **params}
            
            # クラス重みを計算
            ratio = (y_train == 0).sum() / (y_train == 1).sum()
            full_params['scale_pos_weight'] = ratio
            
            model = lgb.LGBMClassifier(**full_params, n_estimators=300)
            # ここでエンコード済みのデータを使用！
            model.fit(X_train_encoded, y_train)
            
            self.models.append(model)
            valid_pred = model.predict_proba(X_valid_encoded)[:, 1]
            valid_predictions.append(valid_pred)
            
            auc = roc_auc_score(y_valid, valid_pred)
            print(f"モデル{i+1} AUC: {auc:.4f}")
        
        # アンサンブル予測
        ensemble_pred = np.mean(valid_predictions, axis=0)
        ensemble_auc = roc_auc_score(y_valid, ensemble_pred)
        print(f"\nアンサンブルAUC: {ensemble_auc:.4f}")
        
        # 特徴量重要度を集計
        self._aggregate_feature_importance(X_train_encoded.columns)
        
        return ensemble_pred
    
    def predict_proba(self, X):
        """アンサンブル予測"""
        # カテゴリカル特徴量をエンコード
        X_encoded = X.copy()
        categorical_cols = X_encoded.select_dtypes(include=['object']).columns
        
        for col in categorical_cols:
            if col in self.label_encoders:
                # 未知のカテゴリは-1に置換
                encoded_values = []
                for val in X_encoded[col].fillna('missing'):
                    if val in self.label_encoders[col].classes_:
                        encoded_values.append(self.label_encoders[col].transform([val])[0])
                    else:
                        encoded_values.append(-1)
                X_encoded[col] = encoded_values
        
        predictions = []
        for model in self.models:
            pred = model.predict_proba(X_encoded)[:, 1]
            predictions.append(pred)
        return np.mean(predictions, axis=0)
    
    def _aggregate_feature_importance(self, feature_names):
        """特徴量重要度を集計"""
        importance_dict = {}
        
        for model in self.models:
            importances = model.feature_importances_
            for feat, imp in zip(feature_names, importances):
                if feat not in importance_dict:
                    importance_dict[feat] = []
                importance_dict[feat].append(imp)
        
        # 平均を計算
        self.feature_importance_df = pd.DataFrame([
            {'feature': feat, 'importance': np.mean(imps)}
            for feat, imps in importance_dict.items()
        ]).sort_values('importance', ascending=False)
    
    def find_optimal_thresholds(self, y_true, y_pred):
        """複数の閾値を見つける（高確率、中確率、低確率）"""
        precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred)
        
        # 高精度閾値（精度重視）
        high_precision_idx = np.where(precisions[:-1] >= 0.6)[0]
        if len(high_precision_idx) > 0:
            high_threshold = thresholds[high_precision_idx[0]]
        else:
            high_threshold = 0.8
            
        # バランス閾値（F値最大）
        f_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1])
        balanced_threshold = thresholds[np.argmax(f_scores)]
        
        # 低閾値（再現率重視）
        low_recall_idx = np.where(recalls[:-1] >= 0.6)[0]
        if len(low_recall_idx) > 0:
            low_threshold = thresholds[low_recall_idx[-1]]
        else:
            low_threshold = 0.3
            
        return {
            'high': high_threshold,
            'balanced': balanced_threshold,
            'low': low_threshold
        }

## 2. 高度な賭け方戦略クラス

In [29]:
class AdvancedBettingStrategy:
    """期待値ベースの高度な賭け方戦略"""
    
    def __init__(self):
        self.betting_records = []
        
    def calculate_expected_value(self, prob, odds):
        """期待値を計算"""
        return prob * odds - 1
    
    def kelly_criterion(self, prob, odds, fraction=0.25):
        """ケリー基準による賭け金比率（保守的に1/4ケリー）"""
        if odds <= 1:
            return 0
        q = 1 - prob
        kelly = (prob * odds - q) / odds
        return max(0, kelly * fraction)
    
    def select_bets(self, race_predictions, thresholds):
        """レースごとに最適な賭け方を選択"""
        betting_plan = []
        
        for race_id, horses in race_predictions.items():
            # 各馬の予測確率でソート
            sorted_horses = sorted(horses, key=lambda x: x['prob'], reverse=True)
            
            race_bets = {
                'race_id': race_id,
                'bets': []
            }
            
            # 1. 単勝・複勝戦略
            for horse in sorted_horses[:3]:  # 上位3頭
                if horse['prob'] >= thresholds['high']:
                    # 高確率馬には単勝・複勝両方
                    race_bets['bets'].append({
                        'type': '単勝',
                        'horses': [horse['horse_num']],
                        'confidence': 'high',
                        'amount_ratio': self.kelly_criterion(horse['prob'], horse['odds'])
                    })
                    race_bets['bets'].append({
                        'type': '複勝',
                        'horses': [horse['horse_num']],
                        'confidence': 'high',
                        'amount_ratio': 0.1  # 複勝は固定比率
                    })
                elif horse['prob'] >= thresholds['balanced']:
                    # 中確率馬には複勝のみ
                    race_bets['bets'].append({
                        'type': '複勝',
                        'horses': [horse['horse_num']],
                        'confidence': 'medium',
                        'amount_ratio': 0.05
                    })
            
            # 2. 馬連・ワイド戦略
            high_prob_horses = [h for h in sorted_horses if h['prob'] >= thresholds['balanced']]
            if len(high_prob_horses) >= 2:
                # 上位2頭の馬連
                race_bets['bets'].append({
                    'type': '馬連',
                    'horses': [high_prob_horses[0]['horse_num'], high_prob_horses[1]['horse_num']],
                    'confidence': 'medium',
                    'amount_ratio': 0.05
                })
                
                # ワイド（より安全）
                for i in range(min(3, len(high_prob_horses))):
                    for j in range(i+1, min(3, len(high_prob_horses))):
                        race_bets['bets'].append({
                            'type': 'ワイド',
                            'horses': [high_prob_horses[i]['horse_num'], high_prob_horses[j]['horse_num']],
                            'confidence': 'medium',
                            'amount_ratio': 0.03
                        })
            
            # 3. 3連複・3連単戦略（期待値が高い場合のみ）
            if len(high_prob_horses) >= 3:
                top3 = high_prob_horses[:3]
                
                # 3連複（順不同）
                combined_prob = top3[0]['prob'] * top3[1]['prob'] * top3[2]['prob'] * 6  # 順列補正
                if combined_prob > 0.05:  # 5%以上の確率
                    race_bets['bets'].append({
                        'type': '3連複',
                        'horses': [h['horse_num'] for h in top3],
                        'confidence': 'low',
                        'amount_ratio': 0.02
                    })
                
                # 3連単（最も自信がある場合のみ）
                if top3[0]['prob'] >= thresholds['high'] and combined_prob > 0.02:
                    # 1着固定、2-3着流し
                    race_bets['bets'].append({
                        'type': '3連単',
                        'horses': [h['horse_num'] for h in top3],
                        'confidence': 'low',
                        'amount_ratio': 0.01
                    })
            
            if race_bets['bets']:
                betting_plan.append(race_bets)
        
        return betting_plan
    
    def calculate_returns(self, betting_plan, payback_data, base_amount=10000):
        """実際の払戻計算"""
        total_investment = 0
        total_return = 0
        returns_by_type = {}
        
        for race_plan in betting_plan:
            race_id = race_plan['race_id']
            
            if race_id not in payback_data.index:
                continue
                
            race_payback = payback_data.loc[race_id]
            
            for bet in race_plan['bets']:
                bet_type = bet['type']
                horses = bet['horses']
                amount = base_amount * bet['amount_ratio']
                
                if amount == 0:
                    continue
                    
                total_investment += amount
                
                # 払戻チェック
                payout = self._check_payout(race_payback, bet_type, horses)
                if payout > 0:
                    return_amount = amount * (payout / 100)  # 100円あたりの払戻
                    total_return += return_amount
                    
                    if bet_type not in returns_by_type:
                        returns_by_type[bet_type] = {'investment': 0, 'return': 0, 'count': 0, 'hits': 0}
                    returns_by_type[bet_type]['return'] += return_amount
                    returns_by_type[bet_type]['hits'] += 1
                else:
                    if bet_type not in returns_by_type:
                        returns_by_type[bet_type] = {'investment': 0, 'return': 0, 'count': 0, 'hits': 0}
                
                returns_by_type[bet_type]['investment'] += amount
                returns_by_type[bet_type]['count'] += 1
        
        return {
            'total_investment': total_investment,
            'total_return': total_return,
            'total_roi': (total_return / total_investment * 100) if total_investment > 0 else 0,
            'by_type': returns_by_type
        }
    
    def _check_payout(self, race_payback, bet_type, horses):
        """払戻チェック"""
        # 払戻データの列名マッピング
        column_map = {
            '単勝': '単勝',
            '複勝': '複勝',
            '馬連': '馬連',
            'ワイド': 'ワイド',
            '3連複': '三連複',
            '3連単': '三連単'
        }
        
        if bet_type not in column_map:
            return 0
            
        col_name = column_map[bet_type]
        if col_name not in race_payback or not isinstance(race_payback[col_name], list):
            return 0
            
        payout_data = race_payback[col_name]
        
        # 馬番の組み合わせを文字列に変換
        if bet_type in ['単勝', '複勝']:
            horse_str = str(horses[0])
        elif bet_type in ['馬連', 'ワイド']:
            horse_str = '-'.join(sorted([str(h) for h in horses]))
        else:  # 3連複、3連単
            if bet_type == '3連複':
                horse_str = '-'.join(sorted([str(h) for h in horses]))
            else:  # 3連単
                horse_str = '-'.join([str(h) for h in horses])
        
        # 払戻データから該当する組み合わせを探す
        for i in range(0, len(payout_data), 2):
            if i+1 < len(payout_data):
                if payout_data[i] == horse_str or (bet_type == 'ワイド' and self._is_wide_match(payout_data[i], horses)):
                    return int(payout_data[i+1].replace(',', ''))
        
        return 0
    
    def _is_wide_match(self, payout_str, horses):
        """ワイドの払戻チェック（順不同）"""
        payout_horses = set(payout_str.split('-'))
        bet_horses = set([str(h) for h in horses])
        return payout_horses == bet_horses

## 3. データ読み込みと前処理

In [ ]:
# データ読み込み
data_path = 'encoded/2022_2023encoded_data.csv'

if os.path.exists(data_path):
    print(f"データ読み込み中: {data_path}")
    data = pd.read_csv(data_path)
    
    # 日付でソート
    data = data.sort_values('日付').reset_index(drop=True)
    
    # 着順を二値化
    data['target'] = data['着順'].map(lambda x: 1 if x < 4 else 0)
    
    # データを3分割（訓練：検証：テスト = 6:2:2）
    n = len(data)
    train_end = int(n * 0.6)
    valid_end = int(n * 0.8)
    
    train_data = data.iloc[:train_end].copy()
    valid_data = data.iloc[train_end:valid_end].copy()
    test_data = data.iloc[valid_end:].copy()
    
    print(f"\nデータ分割:")
    print(f"学習: {len(train_data)}件 ({train_data['日付'].min()} 〜 {train_data['日付'].max()})")
    print(f"検証: {len(valid_data)}件 ({valid_data['日付'].min()} 〜 {valid_data['日付'].max()})")
    print(f"テスト: {len(test_data)}件 ({test_data['日付'].min()} 〜 {test_data['日付'].max()})")
else:
    print(f"エラー: {data_path} が見つかりません")

In [ ]:
# 改善モデルの準備
predictor = ImprovedKeibaPredictor()

# 追加特徴量の作成
print("\n追加特徴量を作成中...")
train_data = predictor.create_additional_features(train_data)
valid_data = predictor.create_additional_features(valid_data)
test_data = predictor.create_additional_features(test_data)

# 特徴量を準備
exclude_cols = ['着順', 'target', 'オッズ', '人気', '上がり', '走破時間', '通過順']
feature_cols = [col for col in train_data.columns if col not in exclude_cols]

X_train = train_data[feature_cols]
y_train = train_data['target']
X_valid = valid_data[feature_cols]
y_valid = valid_data['target']
X_test = test_data[feature_cols]
y_test = test_data['target']

print(f"特徴量数: {len(feature_cols)}")

## 4. アンサンブルモデルの学習

In [ ]:
# アンサンブルモデルの学習
print("アンサンブルモデルを学習中...")
valid_pred = predictor.train_ensemble(X_train, y_train, X_valid, y_valid, n_models=3)

# 最適閾値の探索
thresholds = predictor.find_optimal_thresholds(y_valid, valid_pred)
print(f"\n閾値:")
print(f"高確率: {thresholds['high']:.4f}")
print(f"バランス: {thresholds['balanced']:.4f}")
print(f"低確率: {thresholds['low']:.4f}")

## 5. テストデータでの評価

In [ ]:
# テストデータで予測
test_pred = predictor.predict_proba(X_test)
test_auc = roc_auc_score(y_test, test_pred)

print(f"\nテストAUC: {test_auc:.4f}")

# 各閾値での性能
for name, threshold in thresholds.items():
    pred_binary = (test_pred >= threshold).astype(int)
    TP = ((y_test == 1) & (pred_binary == 1)).sum()
    FP = ((y_test == 0) & (pred_binary == 1)).sum()
    
    if TP + FP > 0:
        precision = TP / (TP + FP)
        print(f"\n{name}閾値 ({threshold:.4f}):")
        print(f"  予測数: {TP + FP}")
        print(f"  的中率: {precision:.2%}")

## 6. 高度な賭け方戦略の実行

In [ ]:
# レースごとの予測を整理
print("\nレースごとの予測を整理中...")
race_predictions = {}

# enumerate を使用して正しいインデックスを取得
for idx, (_, row) in enumerate(test_data.iterrows()):
    race_id = str(int(float(row['race_id'])))
    
    if race_id not in race_predictions:
        race_predictions[race_id] = []
    
    race_predictions[race_id].append({
        'horse_num': str(int(float(row['馬番']))),
        'prob': test_pred[idx],
        'odds': row['オッズ'] if 'オッズ' in row else 10.0  # オッズがない場合はデフォルト値
    })

print(f"テストデータのレース数: {len(race_predictions)}")

In [ ]:
# 賭け方戦略の実行
betting_strategy = AdvancedBettingStrategy()
betting_plan = betting_strategy.select_bets(race_predictions, thresholds)

# 賭けの統計
bet_counts = {}
for race in betting_plan:
    for bet in race['bets']:
        bet_type = bet['type']
        if bet_type not in bet_counts:
            bet_counts[bet_type] = 0
        bet_counts[bet_type] += 1

print("\n賭けの内訳:")
for bet_type, count in sorted(bet_counts.items()):
    print(f"{bet_type}: {count}件")
print(f"\n合計レース数: {len(betting_plan)}")

In [ ]:
# 払戻データの読み込みと計算
print("\n払戻計算中...")

# テストデータの年を特定
test_years = pd.to_datetime(test_data['日付']).dt.year.unique()
print(f"テストデータの年: {sorted(test_years)}")

# 払戻データを読み込み
payback_dfs = []
for year in test_years:
    for ext in ['.xlsx', '.csv']:  # 両方の拡張子を試す
        payback_path = f'payback/{year}{ext}'
        if os.path.exists(payback_path):
            try:
                if ext == '.xlsx':
                    df = pd.read_excel(payback_path, dtype={'race_id': str})
                else:
                    df = pd.read_csv(payback_path, encoding='SHIFT-JIS', dtype={'race_id': str})
                df['race_id'] = df['race_id'].str.replace(r'\.0$', '', regex=True)
                payback_dfs.append(df)
                print(f"{year}年の払戻データを読み込みました")
                break
            except Exception as e:
                print(f"読み込みエラー: {e}")

if payback_dfs:
    payback_df = pd.concat(payback_dfs, ignore_index=True)
    payback_df.set_index('race_id', inplace=True)
    
    # 払戻データの変換
    payout_cols = ['単勝', '複勝', '馬連', 'ワイド', '三連複', '三連単']
    for col in payout_cols:
        if col in payback_df.columns:
            payback_df[col] = payback_df[col].apply(
                lambda x: ast.literal_eval(x) if pd.notna(x) and str(x).strip().startswith('[') else []
            )
    
    # 回収率計算
    results = betting_strategy.calculate_returns(betting_plan, payback_df, base_amount=100)
    
    print("\n=== 総合結果 ===")
    print(f"総投資額: {results['total_investment']:,.0f}円")
    print(f"総払戻額: {results['total_return']:,.0f}円")
    print(f"収支: {results['total_return'] - results['total_investment']:+,.0f}円")
    print(f"回収率: {results['total_roi']:.2f}%")
    
    print("\n=== 賭け方別の結果 ===")
    for bet_type, stats in results['by_type'].items():
        if stats['count'] > 0:
            roi = (stats['return'] / stats['investment'] * 100) if stats['investment'] > 0 else 0
            hit_rate = (stats['hits'] / stats['count'] * 100) if stats['count'] > 0 else 0
            print(f"\n{bet_type}:")
            print(f"  賭け数: {stats['count']}回")
            print(f"  的中数: {stats['hits']}回 ({hit_rate:.1f}%)")
            print(f"  投資額: {stats['investment']:,.0f}円")
            print(f"  払戻額: {stats['return']:,.0f}円")
            print(f"  回収率: {roi:.2f}%")
else:
    print("払戻データが見つかりません")

## 7. 特徴量重要度の可視化

In [ ]:
# Display feature importance
if predictor.feature_importance_df is not None:
    print("\n=== Top 20 Important Features ===")
    top_features = predictor.feature_importance_df.head(20)
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title('Top 20 Feature Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    for i, row in top_features.iterrows():
        print(f"{i+1:2d}. {row['feature']:<30} {row['importance']:.0f}")

## 8. 改善効果のまとめ

In [ ]:
print("\n=== 改善のまとめ ===")
print("\n1. モデル改善:")
print("   - アンサンブル学習による安定性向上")
print("   - 追加特徴量（調子指標、距離適性、重賞実績など）")
print("   - 複数閾値による柔軟な予測")

print("\n2. 賭け方戦略:")
print("   - 期待値ベースの賭け金配分（ケリー基準）")
print("   - 確率に応じた賭け方の使い分け")
print("   - 3連単・3連複を含む高配当狙い")

print("\n3. リスク管理:")
print("   - 保守的なケリー基準（1/4）")
print("   - 分散投資による安定化")
print("   - 高確率馬への重点投資")

if 'results' in locals():
    print("\n4. 期待される効果:")
    if results['total_roi'] > 100:
        print(f"   ✅ 回収率 {results['total_roi']:.2f}% を達成！")
    else:
        print(f"   ⚠️ 回収率は {results['total_roi']:.2f}% （更なる改善が必要）")

In [ ]:
# アンサンブルモデルの学習
print("アンサンブルモデルを学習中...")
valid_pred = predictor.train_ensemble(X_train, y_train, X_valid, y_valid, n_models=3)

# 最適閾値の探索
thresholds = predictor.find_optimal_thresholds(y_valid, valid_pred)
print(f"\n閾値:")
print(f"高確率: {thresholds['high']:.4f}")
print(f"バランス: {thresholds['balanced']:.4f}")
print(f"低確率: {thresholds['low']:.4f}")

In [ ]:
# predictorを再初期化
predictor = ImprovedKeibaPredictor()

# デバッグ: X_trainの中身を確認
print("X_trainのカラム数:", len(X_train.columns))
print("カテゴリカル特徴量:", X_train.select_dtypes(include=['object']).columns.tolist())
print("\n騎手_馬場のサンプル:")
if '騎手_馬場' in X_train.columns:
    print(X_train['騎手_馬場'].head())

In [ ]:
# アンサンブルモデルの学習
print("アンサンブルモデルを学習中...")
valid_pred = predictor.train_ensemble(X_train, y_train, X_valid, y_valid, n_models=3)

# 最適閾値の探索
thresholds = predictor.find_optimal_thresholds(y_valid, valid_pred)
print(f"\n閾値:")
print(f"高確率: {thresholds['high']:.4f}")
print(f"バランス: {thresholds['balanced']:.4f}")
print(f"低確率: {thresholds['low']:.4f}")

In [ ]:
# Visualization: ROI and hit rate by betting type
if 'results_df' in locals() and len(results_df) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # ROI bar chart
    ax1.barh(range(len(results_df)), 
             results_df['回収率(%)'], 
             color=['green' if x > 100 else 'red' for x in results_df['回収率(%)']])
    ax1.set_yticks(range(len(results_df)))
    ax1.set_yticklabels(results_df['賭け方'] + ' (' + results_df['サブタイプ'] + ')')
    ax1.axvline(x=100, color='black', linestyle='--', alpha=0.5)
    ax1.set_xlabel('ROI (%)')
    ax1.set_title('ROI by Betting Type')
    ax1.grid(True, alpha=0.3)
    
    # Hit rate vs bet count scatter plot
    scatter = ax2.scatter(results_df['的中率(%)'], 
                         results_df['回収率(%)'], 
                         s=results_df['賭け数']*2,
                         alpha=0.6)
    ax2.axhline(y=100, color='black', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Hit Rate (%)')
    ax2.set_ylabel('ROI (%)')
    ax2.set_title('Hit Rate vs ROI (circle size = number of bets)')
    ax2.grid(True, alpha=0.3)
    
    # Add labels
    for _, row in results_df.iterrows():
        bet_type_map = {
            '単勝': 'Win',
            '複勝': 'Place',
            '馬連': 'Exacta',
            '馬単': 'Quinella',
            'ワイド': 'Wide',
            '3連複': 'Trifecta',
            '3連単': 'Trio'
        }
        subtype_map = {
            'ストレート': 'Straight',
            'BOX': 'Box',
            '1着固定流し': '1st Fixed',
            '1-2着固定流し': '1-2 Fixed',
            'フォーメーション': 'Formation'
        }
        
        eng_type = bet_type_map.get(row['賭け方'], row['賭け方'])
        eng_subtype = subtype_map.get(row['サブタイプ'], row['サブタイプ'])
        
        ax2.annotate(f"{eng_type}({eng_subtype})", 
                    (row['的中率(%)'], row['回収率(%)']), 
                    fontsize=8, alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Heatmap: ROI by bet type
    pivot_data = results_df.pivot_table(
        values='回収率(%)', 
        index='賭け方', 
        columns='サブタイプ', 
        fill_value=0
    )
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot_data, 
                annot=True, 
                fmt='.1f', 
                cmap='RdYlGn', 
                center=100,
                cbar_kws={'label': 'ROI (%)'},
                linewidths=1)
    plt.title('ROI Heatmap by Bet Type × Subtype')
    plt.tight_layout()
    plt.show()

In [ ]:
# エンコード済みのデータでLightGBMを学習
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# パラメータ設定
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': -1,
    'random_state': 42,
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'n_estimators': 300
}

# クラス重みを計算
ratio = (y_train == 0).sum() / (y_train == 1).sum()
params['scale_pos_weight'] = ratio

# モデル学習
print("テストモデルを学習中...")
model = lgb.LGBMClassifier(**params)
model.fit(X_train_encoded, y_train)

# 検証データで予測
valid_pred = model.predict_proba(X_valid_encoded)[:, 1]
auc = roc_auc_score(y_valid, valid_pred)
print(f"AUC: {auc:.4f}")

print("\n学習成功！")

In [ ]:
# predictorを再初期化
predictor = ImprovedKeibaPredictor()

# アンサンブルモデルの学習
print("アンサンブルモデルを学習中...")
valid_pred = predictor.train_ensemble(X_train, y_train, X_valid, y_valid, n_models=3)

# 最適閾値の探索
thresholds = predictor.find_optimal_thresholds(y_valid, valid_pred)
print(f"\n閾値:")
print(f"高確率: {thresholds['high']:.4f}")
print(f"バランス: {thresholds['balanced']:.4f}")
print(f"低確率: {thresholds['low']:.4f}")

In [ ]:
# predictorを再初期化
predictor = ImprovedKeibaPredictor()

# アンサンブルモデルの学習
print("アンサンブルモデルを学習中...")
valid_pred = predictor.train_ensemble(X_train, y_train, X_valid, y_valid, n_models=3)

# 最適閾値の探索
thresholds = predictor.find_optimal_thresholds(y_valid, valid_pred)
print(f"\n閾値:")
print(f"高確率: {thresholds['high']:.4f}")
print(f"バランス: {thresholds['balanced']:.4f}")
print(f"低確率: {thresholds['low']:.4f}")

In [ ]:
# デバッグ：エラーの原因を特定
print("X_trainの型:", type(X_train))
print("X_trainのカラム数:", len(X_train.columns))

# カテゴリカル特徴量を確認
cat_cols = X_train.select_dtypes(include=['object']).columns
print(f"\nカテゴリカル特徴量: {cat_cols.tolist()}")

# 各カラムのデータ型を確認
print("\nデータ型の確認:")
for col in cat_cols:
    print(f"{col}: {X_train[col].dtype}")

In [ ]:
# セル10の内容を実行
# predictorを再初期化
predictor = ImprovedKeibaPredictor()

# アンサンブルモデルの学習
print("アンサンブルモデルを学習中...")
valid_pred = predictor.train_ensemble(X_train, y_train, X_valid, y_valid, n_models=3)

# 最適閾値の探索
thresholds = predictor.find_optimal_thresholds(y_valid, valid_pred)
print(f"\n閾値:")
print(f"高確率: {thresholds['high']:.4f}")
print(f"バランス: {thresholds['balanced']:.4f}")
print(f"低確率: {thresholds['low']:.4f}")

In [ ]:
# ImprovedKeibaPredictor クラスの train_ensemble メソッドを確認
import inspect
print("train_ensemble メソッドのソースコード (抜粋):")
source = inspect.getsource(ImprovedKeibaPredictor.train_ensemble)
# model.fit の行を探す
for i, line in enumerate(source.split('\n')):
    if 'model.fit' in line:
        print(f"Line {i}: {line.strip()}")

In [ ]:
# 改善モデルの準備
predictor = ImprovedKeibaPredictor()

# 追加特徴量の作成
print("\n追加特徴量を作成中...")
train_data = predictor.create_additional_features(train_data)
valid_data = predictor.create_additional_features(valid_data)
test_data = predictor.create_additional_features(test_data)

# 騎手_馬場カラムを手動でエンコード
from sklearn.preprocessing import LabelEncoder
if '騎手_馬場' in train_data.columns:
    print("騎手_馬場カラムをエンコード中...")
    le = LabelEncoder()
    # 全データを結合してフィット
    all_values = pd.concat([train_data['騎手_馬場'], valid_data['騎手_馬場'], test_data['騎手_馬場']]).fillna('missing')
    le.fit(all_values)
    
    # 各データセットでエンコード
    train_data['騎手_馬場'] = le.transform(train_data['騎手_馬場'].fillna('missing'))
    valid_data['騎手_馬場'] = le.transform(valid_data['騎手_馬場'].fillna('missing'))
    test_data['騎手_馬場'] = le.transform(test_data['騎手_馬場'].fillna('missing'))
    
    print(f"騎手_馬場のユニーク値数: {len(le.classes_)}")

# 特徴量を準備
exclude_cols = ['着順', 'target', 'オッズ', '人気', '上がり', '走破時間', '通過順']
feature_cols = [col for col in train_data.columns if col not in exclude_cols]

X_train = train_data[feature_cols]
y_train = train_data['target']
X_valid = valid_data[feature_cols]
y_valid = valid_data['target']
X_test = test_data[feature_cols]
y_test = test_data['target']

print(f"特徴量数: {len(feature_cols)}")

# カテゴリカル特徴量が残っていないか確認
cat_cols = X_train.select_dtypes(include=['object']).columns
print(f"残っているカテゴリカル特徴量: {cat_cols.tolist()}")

In [ ]:
# アンサンブルモデルの学習
print("アンサンブルモデルを学習中...")
valid_pred = predictor.train_ensemble(X_train, y_train, X_valid, y_valid, n_models=3)

# 最適閾値の探索
thresholds = predictor.find_optimal_thresholds(y_valid, valid_pred)
print(f"\n閾値:")
print(f"高確率: {thresholds['high']:.4f}")
print(f"バランス: {thresholds['balanced']:.4f}")
print(f"低確率: {thresholds['low']:.4f}")

In [ ]:
# テストデータで予測
test_pred = predictor.predict_proba(X_test)
test_auc = roc_auc_score(y_test, test_pred)

print(f"\nテストAUC: {test_auc:.4f}")

# 各閾値での性能
for name, threshold in thresholds.items():
    pred_binary = (test_pred >= threshold).astype(int)
    TP = ((y_test == 1) & (pred_binary == 1)).sum()
    FP = ((y_test == 0) & (pred_binary == 1)).sum()
    
    if TP + FP > 0:
        precision = TP / (TP + FP)
        print(f"\n{name}閾値 ({threshold:.4f}):")
        print(f"  予測数: {TP + FP}")
        print(f"  的中率: {precision:.2%}")

In [ ]:
# レースごとの予測を整理
print("\nレースごとの予測を整理中...")
race_predictions = {}

for idx, row in test_data.iterrows():
    race_id = str(int(float(row['race_id'])))
    
    if race_id not in race_predictions:
        race_predictions[race_id] = []
    
    race_predictions[race_id].append({
        'horse_num': str(int(float(row['馬番']))),
        'prob': test_pred[idx],
        'odds': row['オッズ'] if 'オッズ' in row else 10.0  # オッズがない場合はデフォルト値
    })

print(f"テストデータのレース数: {len(race_predictions)}")

In [ ]:
# レースごとの予測を整理（インデックスをリセット）
print("\nレースごとの予測を整理中...")
race_predictions = {}

# test_dataのインデックスをリセット
test_data_reset = test_data.reset_index(drop=True)

for idx in range(len(test_data_reset)):
    row = test_data_reset.iloc[idx]
    race_id = str(int(float(row['race_id'])))
    
    if race_id not in race_predictions:
        race_predictions[race_id] = []
    
    race_predictions[race_id].append({
        'horse_num': str(int(float(row['馬番']))),
        'prob': test_pred[idx],
        'odds': row['オッズ'] if 'オッズ' in row else 10.0  # オッズがない場合はデフォルト値
    })

print(f"テストデータのレース数: {len(race_predictions)}")

In [ ]:
# 賭け方戦略の実行
betting_strategy = AdvancedBettingStrategy()
betting_plan = betting_strategy.select_bets(race_predictions, thresholds)

# 賭けの統計
bet_counts = {}
for race in betting_plan:
    for bet in race['bets']:
        bet_type = bet['type']
        if bet_type not in bet_counts:
            bet_counts[bet_type] = 0
        bet_counts[bet_type] += 1

print("\n賭けの内訳:")
for bet_type, count in sorted(bet_counts.items()):
    print(f"{bet_type}: {count}件")
print(f"\n合計レース数: {len(betting_plan)}")

In [ ]:
# 払戻データの読み込みと計算
print("\n払戻計算中...")

# テストデータの年を特定
test_years = pd.to_datetime(test_data['日付']).dt.year.unique()
print(f"テストデータの年: {sorted(test_years)}")

# 払戻データを読み込み
payback_dfs = []
for year in test_years:
    for ext in ['.xlsx', '.csv']:  # 両方の拡張子を試す
        payback_path = f'payback/{year}{ext}'
        if os.path.exists(payback_path):
            try:
                if ext == '.xlsx':
                    df = pd.read_excel(payback_path, dtype={'race_id': str})
                else:
                    df = pd.read_csv(payback_path, encoding='SHIFT-JIS', dtype={'race_id': str})
                df['race_id'] = df['race_id'].str.replace(r'\.0$', '', regex=True)
                payback_dfs.append(df)
                print(f"{year}年の払戻データを読み込みました")
                break
            except Exception as e:
                print(f"読み込みエラー: {e}")

if payback_dfs:
    payback_df = pd.concat(payback_dfs, ignore_index=True)
    payback_df.set_index('race_id', inplace=True)
    
    # 払戻データの変換
    payout_cols = ['単勝', '複勝', '馬連', 'ワイド', '三連複', '三連単']
    for col in payout_cols:
        if col in payback_df.columns:
            payback_df[col] = payback_df[col].apply(
                lambda x: ast.literal_eval(x) if pd.notna(x) and str(x).strip().startswith('[') else []
            )
    
    # 回収率計算
    results = betting_strategy.calculate_returns(betting_plan, payback_df, base_amount=100)
    
    print("\n=== 総合結果 ===")
    print(f"総投資額: {results['total_investment']:,.0f}円")
    print(f"総払戻額: {results['total_return']:,.0f}円")
    print(f"収支: {results['total_return'] - results['total_investment']:+,.0f}円")
    print(f"回収率: {results['total_roi']:.2f}%")
    
    print("\n=== 賭け方別の結果 ===")
    for bet_type, stats in results['by_type'].items():
        if stats['count'] > 0:
            roi = (stats['return'] / stats['investment'] * 100) if stats['investment'] > 0 else 0
            hit_rate = (stats['hits'] / stats['count'] * 100) if stats['count'] > 0 else 0
            print(f"\n{bet_type}:")
            print(f"  賭け数: {stats['count']}回")
            print(f"  的中数: {stats['hits']}回 ({hit_rate:.1f}%)")
            print(f"  投資額: {stats['investment']:,.0f}円")
            print(f"  払戻額: {stats['return']:,.0f}円")
            print(f"  回収率: {roi:.2f}%")
else:
    print("払戻データが見つかりません")

In [ ]:
# 日付のデバッグ
print("test_dataの日付列の最初の5行:")
print(test_data['日付'].head())
print(f"\nデータ型: {test_data['日付'].dtype}")
print(f"最小値: {test_data['日付'].min()}")
print(f"最大値: {test_data['日付'].max()}")

# 正しい年を取得する方法を試す
# 日付が数値の場合、それをdatetime型に変換
import datetime

def convert_to_year(date_val):
    try:
        # 日付が数値の場合（Excelの日付形式）
        if isinstance(date_val, (int, float)):
            # Excelの日付形式から変換（1900年1月1日を基準）
            base_date = datetime.datetime(1900, 1, 1)
            actual_date = base_date + datetime.timedelta(days=int(date_val) - 2)  # Excelは1900年を閏年として扱うバグがあるため-2
            return actual_date.year
        else:
            # 文字列の場合
            return pd.to_datetime(date_val).year
    except:
        return None

# 年を正しく取得
test_years_correct = test_data['日付'].apply(convert_to_year).dropna().unique()
print(f"\n正しいテストデータの年: {sorted(test_years_correct)}")

In [ ]:
# 元のCSVファイルから日付の情報を確認
original_data = pd.read_csv('encoded/2022_2023encoded_data.csv')
print("元データの日付列の情報:")
print(f"データ型: {original_data['日付'].dtype}")
print(f"最初の5行: {original_data['日付'].head()}")
print(f"最小値: {original_data['日付'].min()}")
print(f"最大値: {original_data['日付'].max()}")

# race_idから年を推測
print("\nrace_idから年を推測:")
sample_race_ids = test_data['race_id'].head(10)
print(f"サンプルrace_id: {sample_race_ids.tolist()}")

# race_idの最初の4文字が年を表している可能性
test_years_from_race_id = test_data['race_id'].astype(str).str[:4].unique()
print(f"race_idから推測される年: {sorted(test_years_from_race_id)}")

In [ ]:
# 払戻データの読み込みと計算（修正版）
print("\n払戻計算中...")

# race_idから年を取得
test_years = test_data['race_id'].astype(str).str[:4].unique()
print(f"テストデータの年: {sorted(test_years)}")

# 払戻データを読み込み
payback_dfs = []
for year in test_years:
    for ext in ['.xlsx', '.csv']:  # 両方の拡張子を試す
        payback_path = f'payback/{year}{ext}'
        if os.path.exists(payback_path):
            try:
                if ext == '.xlsx':
                    df = pd.read_excel(payback_path, dtype={'race_id': str})
                else:
                    df = pd.read_csv(payback_path, encoding='SHIFT-JIS', dtype={'race_id': str})
                df['race_id'] = df['race_id'].str.replace(r'\.0$', '', regex=True)
                payback_dfs.append(df)
                print(f"{year}年の払戻データを読み込みました")
                break
            except Exception as e:
                print(f"読み込みエラー: {e}")

if payback_dfs:
    payback_df = pd.concat(payback_dfs, ignore_index=True)
    payback_df.set_index('race_id', inplace=True)
    
    # 払戻データの変換
    payout_cols = ['単勝', '複勝', '馬連', 'ワイド', '三連複', '三連単']
    for col in payout_cols:
        if col in payback_df.columns:
            payback_df[col] = payback_df[col].apply(
                lambda x: ast.literal_eval(x) if pd.notna(x) and str(x).strip().startswith('[') else []
            )
    
    # 回収率計算
    results = betting_strategy.calculate_returns(betting_plan, payback_df, base_amount=100)
    
    print("\n=== 総合結果 ===")
    print(f"総投資額: {results['total_investment']:,.0f}円")
    print(f"総払戻額: {results['total_return']:,.0f}円")
    print(f"収支: {results['total_return'] - results['total_investment']:+,.0f}円")
    print(f"回収率: {results['total_roi']:.2f}%")
    
    print("\n=== 賭け方別の結果 ===")
    for bet_type, stats in results['by_type'].items():
        if stats['count'] > 0:
            roi = (stats['return'] / stats['investment'] * 100) if stats['investment'] > 0 else 0
            hit_rate = (stats['hits'] / stats['count'] * 100) if stats['count'] > 0 else 0
            print(f"\n{bet_type}:")
            print(f"  賭け数: {stats['count']}回")
            print(f"  的中数: {stats['hits']}回 ({hit_rate:.1f}%)")
            print(f"  投資額: {stats['investment']:,.0f}円")
            print(f"  払戻額: {stats['return']:,.0f}円")
            print(f"  回収率: {roi:.2f}%")
else:
    print("払戻データが見つかりません")

In [ ]:
# 特徴量重要度の表示
if predictor.feature_importance_df is not None:
    print("\n=== 重要な特徴量トップ20 ===")
    top_features = predictor.feature_importance_df.head(20)
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('重要度')
    plt.title('特徴量重要度トップ20')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    for i, row in top_features.iterrows():
        print(f"{i+1:2d}. {row['feature']:<30} {row['importance']:.0f}")

In [ ]:
print("\n=== 改善のまとめ ===")
print("\n1. モデル改善:")
print("   - アンサンブル学習による安定性向上")
print("   - 追加特徴量（調子指標、距離適性、重賞実績など）")
print("   - 複数閾値による柔軟な予測")

print("\n2. 賭け方戦略:")
print("   - 期待値ベースの賭け金配分（ケリー基準）")
print("   - 確率に応じた賭け方の使い分け")
print("   - 3連単・3連複を含む高配当狙い")

print("\n3. リスク管理:")
print("   - 保守的なケリー基準（1/4）")
print("   - 分散投資による安定化")
print("   - 高確率馬への重点投資")

if 'results' in locals():
    print("\n4. 期待される効果:")
    if results['total_roi'] > 100:
        print(f"   ✅ 回収率 {results['total_roi']:.2f}% を達成！")
    else:
        print(f"   ⚠️ 回収率は {results['total_roi']:.2f}% （更なる改善が必要）")
        
print("\n5. 重要な発見:")
print("   - 騎手の勝率が最も重要な特徴量")
print("   - 過去のパフォーマンス（賞金、着順）が重要")
print("   - 騎手×馬場の相性も一定の重要度を持つ")
print("   - 単勝・複勝が最も安定した回収率を実現")